[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/Copy_of_DNST_CIFAR10_AUGC1.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20
compression = 1
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
from keras.preprocessing.image import ImageDataGenerator #, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(
        zoom_range=0.2,
        horizontal_flip=True)
'''        vertical_flip=True,
        rotation_range=40,
        horizontal_flip=True
     fill_mode='nearest',
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
'''
train_gen = datagen.flow(x_train, y_train, batch_size)
test_gen = datagen.flow(x_test, y_test, batch_size)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
#num_filter = 12
#dropout_rate = 0.2
#l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [33]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_249 (Conv2D)             (None, 32, 32, 20)   540         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_249 (BatchN (None, 32, 32, 20)   80          conv2d_249[0][0]                 
__________________________________________________________________________________________________
activation_249 (Activation)     (None, 32, 32, 20)   0           batch_normalization_249[0][0]    
__________________________________________________________________________________________________
conv2d_250

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [35]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
'''
model.fit_generator(train_gen,
                    epochs=epochs,
                    verbose=1,
#                    steps_per_epoch=len(x_train),
                    validation_data=test_gen)

Epoch 1/50
391/391 [==============================] - 293s 750ms/step - loss: 1.5254 - acc: 0.4378 - val_loss: 1.3454 - val_acc: 0.5239
Epoch 2/50
280/391 [====================>.........] - ETA: 1:12 - loss: 1.1603 - acc: 0.5847

391/391 [==============================] - 270s 690ms/step - loss: 1.1243 - acc: 0.5992 - val_loss: 1.7389 - val_acc: 0.4965
Epoch 3/50
391/391 [==============================] - 270s 690ms/step - loss: 0.9329 - acc: 0.6692 - val_loss: 0.9993 - val_acc: 0.6592
Epoch 4/50
 16/391 [>.............................] - ETA: 4:01 - loss: 0.8555 - acc: 0.6978

391/391 [==============================] - 270s 690ms/step - loss: 0.8253 - acc: 0.7102 - val_loss: 1.6687 - val_acc: 0.5602
Epoch 5/50
320/391 [=======================>......] - ETA: 46s - loss: 0.7602 - acc: 0.7325

391/391 [==============================] - 270s 690ms/step - loss: 0.7554 - acc: 0.7341 - val_loss: 1.3851 - val_acc: 0.6036
Epoch 6/50
391/391 [==============================] - 270s 690ms/step - loss: 0.7024 - acc: 0.7516 - val_loss: 0.7969 - val_acc: 0.7442
Epoch 7/50
 25/391 [>.............................] - ETA: 3:57 - loss: 0.6318 - acc: 0.7816

391/391 [==============================] - 270s 690ms/step - loss: 0.6548 - acc: 0.7705 - val_loss: 1.1368 - val_acc: 0.6677
Epoch 8/50
323/391 [=======================>......] - ETA: 44s - loss: 0.6186 - acc: 0.7811

...

391/391 [==============================] - 271s 692ms/step - loss: 0.2688 - acc: 0.9050 - val_loss: 0.4918 - val_acc: 0.8558
Epoch 45/50
391/391 [==============================] - 270s 691ms/step - loss: 0.2668 - acc: 0.9060 - val_loss: 0.5170 - val_acc: 0.8529
Epoch 46/50
 24/391 [>.............................] - ETA: 3:59 - loss: 0.2547 - acc: 0.9023

391/391 [==============================] - 271s 693ms/step - loss: 0.2584 - acc: 0.9087 - val_loss: 0.4756 - val_acc: 0.8667
Epoch 47/50
321/391 [=======================>......] - ETA: 45s - loss: 0.2591 - acc: 0.9083

391/391 [==============================] - 269s 688ms/step - loss: 0.2594 - acc: 0.9082 - val_loss: 0.4485 - val_acc: 0.8697
Epoch 48/50
391/391 [==============================] - 268s 686ms/step - loss: 0.2508 - acc: 0.9109 - val_loss: 0.6551 - val_acc: 0.8253
Epoch 49/50
 24/391 [>.............................] - ETA: 4:00 - loss: 0.2674 - acc: 0.8981

391/391 [==============================] - 270s 691ms/step - loss: 0.2491 - acc: 0.9116 - val_loss: 0.6546 - val_acc: 0.8259
Epoch 50/50
321/391 [=======================>......] - ETA: 45s - loss: 0.2480 - acc: 0.9134

391/391 [==============================] - 269s 688ms/step - loss: 0.2486 - acc: 0.9126 - val_loss: 0.5593 - val_acc: 0.8522


In [36]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 23s 2ms/step
Test loss: 0.6152515197515488
Test accuracy: 0.8407


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')